### Import TensorFlow and other libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path_to_data = "/content/drive/MyDrive/DATA228/code/data/TextData"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

In [ ]:
# Open and read the file with explicit encoding
artist = str(input('Type in user name')) # getting artist name from user
file_name = artist+".txt"
file_path = path_to_data + "/" + file_name

try:
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        print(content)

except FileNotFoundError:
    print(f"The file {file_path} was not found.")

except Exception as e:
    print(f"An error occurred: {str(e)}")


Type in user namedrake
['yeah uhhuh um uhhuh okay   hello mothafucka hey hi how ya durrn its weezy f baby come to take a shit and urine on these toilet bowl bitches pussy ass niggas stomping on this beat like a motherfucking sigma bad to the bristle hat to the rizzle im so official all i need is a whistle bitch named crystal let her suck my pistol she open up her mouth and then i blow her brains out you aint met a nigga like me you probably never will riders rollin with me like 11 wheels find out where you livin kill you in the kitchen the feds tryin to clip us but we aint even trippin   im goin in im goin in im goin in im goin in and ima go hard and ima go hard and ima go hard and ima go hard bitch im goin in im goin in im goin in im goin in and ima go hard and ima go hard and ima go hard and ima go hard  first off you know what it is if you heard drake making hoes wobble like a bridge in a earthquake never see me out cause i live in my workplace ill give you the business so button up

#### Read the data
#### First, look in the text

In [ ]:
# Read, then decode for py2 compat.
text = open(file_path, 'rb').read().decode(encoding='utf-8')

# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1162118 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

['yeah uhhuh um uhhuh okay   hello mothafucka hey hi how ya durrn its weezy f baby come to take a shit and urine on these toilet bowl bitches pussy ass niggas stomping on this beat like a motherfucking sigma bad to the bristle hat to the rizzle im so


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

42 unique characters


In [ ]:
import string

def find_special_characters(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            special_characters = set(content) - set(string.printable)
            if special_characters:
                print("Special characters found:")
                for char in special_characters:
                    print(f"{char} (Unicode: {ord(char)})")
            else:
                print("No special characters found.")
    except FileNotFoundError:
        print(f"The file {file_path} was not found.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

find_special_characters(file_path)

No special characters found.


Process the text
Vectorize the text
Before training, you need to convert the strings to a numerical representation.

The tf.keras.layers.StringLookup layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

This layer is commonly used in natural language processing tasks where text data needs to be converted into a format that a neural network can process, using integers instead of raw text

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
# NOW ORIGINAL TEXT
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1162118,), dtype=int64, numpy=array([14,  3, 41, ..., 24,  3, 16])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

[
'
y
e
a
h
 
u
h
h


In [ ]:
#  It converts each batch of ID sequences into characters and prints the result for the first batch.
# The use of batches is common in machine learning, where models are often trained on sequences of data rather than individual data points.
seq_length = 50
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'[' b"'" b'y' b'e' b'a' b'h' b' ' b'u' b'h' b'h' b'u' b'h' b' ' b'u'
 b'm' b' ' b'u' b'h' b'h' b'u' b'h' b' ' b'o' b'k' b'a' b'y' b' ' b' '
 b' ' b'h' b'e' b'l' b'l' b'o' b' ' b'm' b'o' b't' b'h' b'a' b'f' b'u'
 b'c' b'k' b'a' b' ' b'h' b'e' b'y' b' ' b'h'], shape=(51,), dtype=string)


In [ ]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b"['yeah uhhuh um uhhuh okay   hello mothafucka hey h"
b'i how ya durrn its weezy f baby come to take a shit'
b' and urine on these toilet bowl bitches pussy ass n'
b'iggas stomping on this beat like a motherfucking si'
b'gma bad to the bristle hat to the rizzle im so offi'


In [ ]:
# This is a common pattern when working with sequential data, like in training a model
# to predict the next element in a sequence.
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
dataset = sequences.map(split_input_target)
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"['yeah uhhuh um uhhuh okay   hello mothafucka hey "
Target: b"'yeah uhhuh um uhhuh okay   hello mothafucka hey h"


Create training batches
prepare the dataset for training by shuffling, batching, and prefetching the data for better performance.

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 50), dtype=tf.int64, name=None), TensorSpec(shape=(64, 50), dtype=tf.int64, name=None))>

In [ ]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

This model has three layers:

tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;
tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)

tf.keras.layers.Dense: The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

This type of architecture is commonly used in natural language processing tasks like text generation or language modeling.
It uses an embedding layer to convert input words into vectors, a GRU layer to capture sequential patterns, and a dense layer to produce the final output(generate logits predicting the log-likelihood of the next character:)

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [ ]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 50, 43) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  11008     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  44075     
                                                                 
Total params: 3993387 (15.23 MB)
Trainable params: 3993387 (15.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# sampling random indices for each prediction in the first sequence of your batch and
# then converting the result to a NumPy array for easier manipulation.
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
sampled_indices

array([24,  0, 33,  2, 36, 13, 39, 28,  7, 34, 15, 14, 10,  6, 42, 10, 21,
       17, 21,  4, 18,  6, 14,  0, 34, 16,  8, 10,  7,  6, 40, 19, 40,  9,
       11, 13, 41, 36, 12, 10,  0, 24, 32,  2, 41,  9, 36,  8, 30,  8])

In [ ]:
# Decode these to see the text predicted by this untrained model:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'yall dont get it do ya type of money everybody act'

Next Char Predictions:
 b'h[UNK]q t9wl3r\\[62z6eae0b2[[UNK]r]4632xcx579yt86[UNK]hp y5t4n4'


Train the model

The loss indicates how well the model's predictions match the actual labels, and minimizing this loss is the objective during the training of a machine learning model.

Adam is an optimization algorithm used to minimize the loss function during training. It adjusts the model's weights to improve its performance.

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 50, 43)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(3.760416, shape=(), dtype=float32)


In [ ]:
tf.exp(example_batch_mean_loss).numpy()

42.966297

In [ ]:
model.compile(optimizer='adam', loss=loss)

this code compiles a neural network model, prepares a directory to save checkpoints, and configures a callback to save the model's weights at the end of each training epoch. Checkpoints are useful for resuming training, evaluating the model's performance at different stages, or deploying the model for inference.

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
356/356 [==============================] - 14s 34ms/step - loss: 2.2595
Epoch 2/20
356/356 [==============================] - 12s 31ms/step - loss: 1.6630
Epoch 3/20
356/356 [==============================] - 14s 33ms/step - loss: 1.4743
Epoch 4/20
356/356 [==============================] - 12s 32ms/step - loss: 1.3648
Epoch 5/20
356/356 [==============================] - 12s 31ms/step - loss: 1.2810
Epoch 6/20
356/356 [==============================] - 13s 34ms/step - loss: 1.2038
Epoch 7/20
356/356 [==============================] - 13s 33ms/step - loss: 1.1267
Epoch 8/20
356/356 [==============================] - 12s 31ms/step - loss: 1.0470
Epoch 9/20
356/356 [==============================] - 13s 31ms/step - loss: 0.9674
Epoch 10/20
356/356 [==============================] - 13s 32ms/step - loss: 0.8908
Epoch 11/20
356/356 [==============================] - 12s 31ms/step - loss: 0.8213
Epoch 12/20
356/356 [==============================] - 11s 30ms/step - loss: 0.7614
E

In [ ]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
        # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
        return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant([artist])
result = [next_char]

spaces = 0

for n in range(500):
  if spaces == 8:
    spaces = 0
    result.append("\n")
  else:
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    if next_char == ' ':
      spaces += 1
    result.append(next_char)


result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

drake sett sitaka    static 13 
mix jones smith toddo  fasta in a 
elea man i got everything that i created 
in anything   im sayin either way 
we bout to happen ayy well see whats 
bout to happen ayy well see whats bout 
to happen ayy well see whats bout to 
happen ayy well see whats bout to happen 
  im making someone with no play 
these other dudds im so coldun in my 
funnythine times spring shorty was birthed in my 
work i go back to have for fucking 
mario way i feel it baby what about 
it damb  

________________________________________________________________________________

Run time: 2.771873950958252


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from difflib import SequenceMatcher
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [ ]:
result_array = result.numpy()[0]
result_string = result_array.decode('utf-8')
result_string.replace("\n", "")

# 1. BLEU Score (for evaluating novelty)
chencherry = SmoothingFunction()
bleu_score = sentence_bleu(content, result_string, smoothing_function=chencherry.method1)

print(f"BLEU Score: {float(bleu_score)}")

BLEU Score: 0.0008124396373713109


In [ ]:
# 2. Sentiment Analysis
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()
sentiment_score = sia.polarity_scores(result_string)['compound']
print(f"Sentiment Score: {sentiment_score}")

Sentiment Score: 0.8225


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
# 3. Rhyme and Rhythm (using SequenceMatcher)
rhyme_similarity = SequenceMatcher(None, result_string, content).ratio()
print(f"Rhyme and Rhythm Similarity: {format(float(rhyme_similarity), 'f')}")

Rhyme and Rhythm Similarity: 0.000045


In [ ]:
# 4. TextBlob for Subjectivity
blob = TextBlob(result_string)
subjectivity_score = blob.sentiment.subjectivity
print(f"Subjectivity Score: {subjectivity_score}")

Subjectivity Score: 0.51875


In [ ]:
nltk.download('punkt')

# 5. Evaluation Against Existing Lyrics (using NLTK's word_tokenize)
reference_tokens = nltk.word_tokenize(content.lower())
generated_tokens = nltk.word_tokenize(result_string.lower())
overlap_score = len(set(reference_tokens) & set(generated_tokens)) / len(set(reference_tokens))
print(f"Overlap Score: {overlap_score}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Overlap Score: 0.004010207801676996
